# 💰 FinMate - Smart Budgeting with 50:30:20 + ML Support


Notebook ini menggabungkan pendekatan *rule-based budgeting* (50:30:20) dengan model *machine learning regression* berbasis TensorFlow untuk memberikan rekomendasi pengeluaran cerdas berdasarkan pendapatan dan kategori yang dimasukkan pengguna.

**Input:**
- Pendapatan bulanan pengguna
- Daftar kategori pengeluaran (bebas)

**Output:**
- Rekomendasi alokasi per kategori (Rupiah)


In [ ]:

import pandas as pd
import numpy as np
import tensorflow as tf
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving student_spending.csv to student_spending.csv


In [ ]:

df = pd.read_csv("student_spending.csv")
df = df.drop(columns=["Unnamed: 0"])

expense_cols = ['tuition', 'housing', 'food', 'transportation', 'books_supplies',
                'entertainment', 'personal_care', 'technology', 'health_wellness', 'miscellaneous']
df["total_spending"] = df[expense_cols].sum(axis=1)


In [ ]:

cat_cols = ['gender', 'year_in_school', 'major', 'preferred_payment_method']
df = pd.get_dummies(df, columns=cat_cols)

# Features
X = df.drop(columns=expense_cols + ['total_spending'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Output: total spending per kategori (multioutput)
y = df[expense_cols]

# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1])  # regresi untuk banyak output
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=0)

model.save("spending_regression_model.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:

# Menyimpan model TensorFlow setelah training
model.save("finmate_model.h5")

# Untuk memuat model di waktu lain:
# from tensorflow.keras.models import load_model
# model = load_model("finmate_model.h5")


In [ ]:

def recommend_budget(income, categories):
    # Proporsi historis per kategori
    proportions = {
        'food': 0.3459,
        'transportation': 0.1707,
        'books_supplies': 0.2393,
        'technology': 0.2441,
        'entertainment': 0.5829,
        'personal_care': 0.4171,
        'health_wellness': 0.5121,
        'miscellaneous': 0.4879
    }

    # Grup kategori
    group_map = {
        'food': 'needs',
        'transportation': 'needs',
        'books_supplies': 'needs',
        'technology': 'needs',
        'entertainment': 'wants',
        'personal_care': 'wants',
        'health_wellness': 'savings',
        'miscellaneous': 'savings'
    }

    group_ratios = {
        'needs': 0.5,
        'wants': 0.3,
        'savings': 0.2
    }

    # Pisahkan kategori yang dipilih per grup
    selected_by_group = {'needs': [], 'wants': [], 'savings': []}
    for cat in categories:
        if cat in group_map and cat in proportions:
            selected_by_group[group_map[cat]].append(cat)

    # Hitung total proporsi untuk normalisasi dalam masing-masing grup
    normalized_props = {}
    for group, cat_list in selected_by_group.items():
        total_prop = sum([proportions[cat] for cat in cat_list])
        if total_prop > 0:
            for cat in cat_list:
                normalized_props[cat] = proportions[cat] / total_prop

    # Hitung alokasi
    budget_allocation = {}
    active_groups = [g for g in selected_by_group if selected_by_group[g]]
    total_ratio = sum([group_ratios[g] for g in active_groups])

    for cat in normalized_props:
        group = group_map[cat]
        ratio = group_ratios[group] / total_ratio  # Normalisasi rasio antar grup yang tersedia
        budget_allocation[cat] = round(income * ratio * normalized_props[cat], 2)

    return budget_allocation


In [ ]:

# Contoh input dari user (bisa diganti)
user_income = 3000000
user_categories = ['food', 'transportation', 'technology', 'books_supplies', 'personal_care', 'entertainment', 'health_wellness', 'miscellaneous']

recommend_budget(user_income, user_categories)


{'food': 518850.0,
 'transportation': 256050.0,
 'technology': 366150.0,
 'books_supplies': 358950.0,
 'personal_care': 375390.0,
 'entertainment': 524610.0,
 'health_wellness': 307260.0,
 'miscellaneous': 292740.0}

In [ ]:
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']